In [1]:
import ndjson
import cv2
import os
import pandas as pd
import numpy as np

# Open and read the NDJSON file
with open('D:\\machine project\\video-input\\videos.ndjson', 'r') as f:
    data = ndjson.load(f)


print(f"Combined records: {len(data)}")


Combined records: 69


In [2]:
df_meta = pd.DataFrame(data)
df_meta.drop(columns=['media_attributes'], inplace=True)
df_meta['data_row'] = df_meta['data_row'].astype(str).str[8:37]
df_meta.rename(columns={'data_row': 'id'}, inplace=True)
df_meta.rename(columns={'metadata_fields': 'emotion'}, inplace=True)
df_meta.head()

,id,emotion
0,cmhpke9p4cjbs0782sob057xx.mp4,[{'value': 'Surprised'}]
1,cmhpke9p4cjbt0782363n2x0u.mp4,[{'value': 'Surprised'}]
2,cmhpke9p4cjbu0782assgc5x9.mp4,[{'value': 'Surprised'}]
3,cmhpke9p4cjbv0782ah5bfrg4.mp4,[{'value': 'Surprised'}]
4,cmhpke9p4cjbw0782wkp7yx3s.mp4,[{'value': 'Surprised'}]


In [4]:
# --- ROBUST DATA CLEANING ---
def clean_emotion_col(x):
    # 1. If it's a list (e.g., ['angry']), grab the first item
    if isinstance(x, list):
        if len(x) > 0:
            x = x[0]
        else:
            return "unknown"

    # 2. If it's a dictionary (e.g., {'label': 'angry'}), grab the value
    if isinstance(x, dict):
        # Try common keys, or just grab the first value found
        x = x.get('label', x.get('emotion', list(x.values())[0]))

    # 3. Convert whatever is left to a string and lower-case it
    text = str(x).lower()
    
    # 4. Remove any leftover garbage characters
    text = text.replace("'", "").replace('"', "").strip()
    return text

# Apply the fix
print("Cleaning data...")
df_meta['emotion'] = df_meta['emotion'].apply(clean_emotion_col)
# --- VERIFY ---
print("\nUnique emotions found:")
print(df_meta['emotion'].unique())

Cleaning data...

Unique emotions found:
['surprised' 'sad' 'neutral' 'happy' 'disgust' 'angry']


In [8]:
import cv2
import os
import pandas as pd
import numpy as np

# --- CONFIGURATION ---
video_folder = "D:\\machine project\\data\\videos"
output_folder = "data/processed_frames_HD" 
img_size = (128, 128) 

# --- FIX IS HERE ---
# 1. Define the path
cascade_path = r"C:\Users\PC\anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml"

# 2. Check if file exists (Crucial because Windows paths can be tricky)
if not os.path.exists(cascade_path):
    print(f"❌ ERROR: Cannot find the Cascade XML at: {cascade_path}")
    print("   Please check the path again.")
    exit()

# 3. Actually LOAD the classifier object
face_cascade = cv2.CascadeClassifier(cascade_path)

# Verify it loaded correctly
if face_cascade.empty():
    print("❌ ERROR: Cascade file found but failed to load. Is it corrupt?")
    exit()
else:
    print(f"✅ Face Detector loaded successfully from: {cascade_path}")


# --- START HARVESTING ---
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

print(f"🚜 Starting HD Harvest from {len(df_meta)} videos...")

count = 0
faces_found_count = 0

for index, row in df_meta.iterrows():
    video_filename = row['id']
    emotion_label = row['emotion']
    
    # Check extension
    if not video_filename.endswith('.mp4'):
        video_filename += ".mp4"
        
    video_path = os.path.join(video_folder, video_filename)
    
    # Skip if video file missing
    if not os.path.exists(video_path):
        continue

    # Create Emotion Folder
    label_path = os.path.join(output_folder, emotion_label)
    os.makedirs(label_path, exist_ok=True)
    
    cap = cv2.VideoCapture(video_path)
    frame_id = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # 1. Convert to Gray (Face detection needs grayscale)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # 2. Detect Faces
        # scaleFactor=1.1 -> Scans image at different sizes (10% increments)
        # minNeighbors=4  -> Higher = fewer false positives, but might miss faces
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
        
        # 3. If a face is found, crop it
        if len(faces) > 0:
            for (x, y, w, h) in faces:
                # Crop: [y_start:y_end, x_start:x_end]
                face_img = frame[y:y+h, x:x+w]
                
                try:
                    # High Quality Resize
                    face_resized = cv2.resize(face_img, img_size, interpolation=cv2.INTER_AREA)
                    
                    # Save
                    clean_name = video_filename.replace('.mp4', '')
                    save_name = f"{clean_name}_f{frame_id}.jpg"
                    cv2.imwrite(os.path.join(label_path, save_name), face_resized)
                    
                    faces_found_count += 1
                except Exception as e:
                    pass # Skip if resize crashes (rare)
                
                # Only take the first face to avoid duplicates/crowd
                break 
        
        frame_id += 1
        count += 1

    cap.release()

print(f"✅ HD Harvest Complete!")
print(f"   Scanned {count} total frames.")
print(f"   Saved {faces_found_count} high-quality face images to '{output_folder}'")

✅ Face Detector loaded successfully from: C:\Users\PC\anaconda3\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml
🚜 Starting HD Harvest from 69 videos...
✅ HD Harvest Complete!
   Scanned 4677 total frames.
   Saved 4669 high-quality face images to 'data/processed_frames_HD'


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# --- CONFIGURATION ---
# We use the HD folder for BOTH training and validation
dataset_path = "data/processed_frames_HD" 
img_height, img_width = 128, 128  # Match the size you harvested!

# 1. LOAD DATA
datagen = ImageDataGenerator(
    rescale=1.0/255,      # Normalize
    validation_split=0.2  # 20% split
)

print(f"📂 Loading Training Data from {dataset_path}...")
train_ds = datagen.flow_from_directory(
    dataset_path,             # FIXED PATH
    target_size=(img_height, img_width), # FIXED SIZE (128x128)
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

print(f"📂 Loading Validation Data from {dataset_path}...")
val_ds = datagen.flow_from_directory(
    dataset_path,             # FIXED PATH
    target_size=(img_height, img_width), # FIXED SIZE (128x128)
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# 2. DEFINE MODEL (Updated for 128x128 input)
model = Sequential([
    # --- CNN PART ---
    # Input Shape is now 128x128x3
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    # Optional: Add a 4th layer since 128x128 is deeper
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    
    # --- FCN PART ---
    Dense(128, activation='relu'),
    Dropout(0.5),
    
    Dense(64, activation='relu'),
    Dropout(0.3),
    
    # Output
    Dense(train_ds.num_classes, activation='softmax')
])

# 3. TRAIN
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("🚀 Starting Training on HD Data...")
history = model.fit(train_ds, epochs=15, validation_data=val_ds)

📂 Loading Training Data from data/processed_frames_HD...
Found 3622 images belonging to 6 classes.
📂 Loading Validation Data from data/processed_frames_HD...
Found 903 images belonging to 6 classes.
🚀 Starting Training on HD Data...


c:\Users\PC\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
114/114 ━━━━━━━━━━━━━━━━━━━━ 27s 227ms/step - accuracy: 0.4412 - loss: 1.4205 - val_accuracy: 0.5194 - val_loss: 1.3153
Epoch 2/15
114/114 ━━━━━━━━━━━━━━━━━━━━ 11s 98ms/step - accuracy: 0.6676 - loss: 0.9010 - val_accuracy: 0.6002 - val_loss: 1.0317
Epoch 3/15
114/114 ━━━━━━━━━━━━━━━━━━━━ 11s 97ms/step - accuracy: 0.7852 - loss: 0.5919 - val_accuracy: 0.6600 - val_loss: 1.0751
Epoch 4/15
114/114 ━━━━━━━━━━━━━━━━━━━━ 11s 94ms/step - accuracy: 0.8490 - loss: 0.4146 - val_accuracy: 0.5703 - val_loss: 1.4805
Epoch 5/15
114/114 ━━━━━━━━━━━━━━━━━━━━ 11s 95ms/step - accuracy: 0.8876 - loss: 0.3203 - val_accuracy: 0.6412 - val_loss: 1.3061
Epoch 6/15
114/114 ━━━━━━━━━━━━━━━━━━━━ 11s 95ms/step - accuracy: 0.9194 - loss: 0.2351 - val_accuracy: 0.5327 - val_loss: 2.0926
Epoch 7/15
114/114 ━━━━━━━━━━━━━━━━━━━━ 11s 97ms/step - accuracy: 0.9224 - loss: 0.2223 - val_accuracy: 0.6711 - val_loss: 1.6405
Epoch 8/15
114/114 ━━━━━━━━━━━━━━━━━━━━ 11s 95ms/step - accuracy: 0.9268 - loss: 0.2050 -

In [1]:
from speechbrain.inference.interfaces import foreign_class
classifier = foreign_class(source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP", pymodule_file="custom_interface.py", classname="CustomEncoderWav2vec2Classifier")
out_prob, score, index, text_lab = classifier.classify_file("speechbrain/emotion-recognition-wav2vec2-IEMOCAP/anger.wav")
print(text_lab)


custom_interface.py: 0.00B [00:00, ?B/s]

c:\Users\PC\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PC\.cache\huggingface\hub\models--speechbrain--emotion-recognition-wav2vec2-IEMOCAP. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


hyperparams.yaml: 0.00B [00:00, ?B/s]

Unexpected exception formatting exception. Falling back to standard exception


c:\Users\PC\anaconda3\Lib\inspect.py:1007: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
Traceback (most recent call last):
  File "c:\Users\PC\anaconda3\Lib\site-packages\speechbrain\integrations\k2_fsa\__init__.py", line 12, in <module>
    import k2  # noqa
    ^^^^^^^^^
ModuleNotFoundError: No module named 'k2'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\PC\anaconda3\Lib\site-packages\speechbrain\utils\importutils.py", line 97, in ensure_module
    self.lazy_module = importlib.import_module(self.target)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\PC\anaconda3\Lib\importlib\__init__.py", line 90, in import_module
    return _bootstra